In [1]:
import torch as t
from config import Config
from model import Transformer 
from tokenizer import Tokenizer
from helpers import extract_answer_from_prediction, add_carry_columns_to_df
import pandas as pd

Using device: cpu
fixed_digit: False
Training for 3000 epochs
Batch size: 256
Learning rate: 0.001
Train on 0.5 of the data
Saving model every 5 epochs


In [10]:
# Load the model
config = Config()
model_path = "saved_runs/variable_digit_add_50/final.pth"
data_path = "saved_runs/variable_digit_add_50/data.csv"
device = config.device
tokenizer = Tokenizer(Config())
model = Transformer(Config())
model.load_state_dict(t.load(model_path, map_location = device)["model"])
model.eval()

C:\Users\aylin\AppData\Local\Temp\ipykernel_13324\670764898.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(model_path, map_location = devic

Transformer(
  (embed): Embed()
  (pos_embed): PosEmbed()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn): HookPoint()
        (hook_attn_pre): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)

In [3]:
def print_output(activation, **kwargs):
    name = kwargs.get('name', 'Unknown')  # Extract 'name' from kwargs, default to 'Unknown' if not provided
    print(f"Output from hook {name}: {activation}\n")
    print("Shape: ", activation.shape)


model.blocks[-1].hook_resid_post.add_hook(print_output)

In [4]:
tokenizer = Tokenizer(Config())

test_sentence = "56+52="
test_sentence_tokenized = tokenizer.tokenize(test_sentence)

pred = model.generate_greedy(test_sentence_tokenized)
answer = extract_answer_from_prediction(pred, tokenizer)
print("Test sentence:", test_sentence)
print("Answer:", answer)

Output from hook blocks.0.hook_resid_post: tensor([[[-1.0167e+00,  1.4733e+00,  1.7920e+00,  6.0812e+00,  5.6519e-01,
           1.4312e+00, -4.4680e+00, -7.2742e-01, -4.0242e+00,  1.0291e+00,
           1.4306e+00, -1.8436e+00,  8.3299e+00,  1.2214e+00, -5.8374e+00,
          -6.1983e-01,  1.2566e+00, -6.6080e-02, -4.9614e+00,  1.8846e+00,
          -1.1839e+00, -3.7025e+00,  2.9835e+00,  2.1988e+00, -3.7544e+00,
           5.2385e-01, -2.8254e+00,  3.6590e-01,  6.0603e+00, -1.9179e-01,
           1.0158e+00, -3.5576e+00, -9.9589e-01,  1.1032e+00, -1.0853e+00,
          -2.7027e+00,  1.4531e+00,  2.6005e+00, -3.1651e+00,  9.3676e-01,
          -1.2512e+00, -9.9380e-01,  1.6172e+00,  2.5258e+00, -2.6167e+00,
           1.8641e+00,  1.9122e+00,  3.0233e-01,  1.0043e+00, -3.0444e+00,
          -1.6027e+00,  5.8922e+00, -5.9778e+00,  1.4248e-03, -8.0163e-01,
           5.8603e+00, -3.8595e-01, -3.5579e-01,  2.9536e+00,  1.9281e+00,
          -6.8833e-02,  3.3695e-01, -2.3256e+00, -1.5062e

In [11]:
add_carry_columns_to_df(data_path)

path saved_runs/variable_digit_add_50/data.csv
